## 多线程爬虫
有些时候，比如下载图片，因为下载图片是一个耗时的操作。如果采用之前那种同步的方式下载。那效率肯定特别慢。这时候我们就可以考虑使用多线程的方式来下载图片。
## 多线程介绍：
多线程是为了同步完成多项任务，通过提高资源使用效率来提高系统的效率。现场是在同一时间需要完成多项任务的时候实现的。最简单的比喻**多线程**就像火车的每一节车厢，而**进程**则是火车。车厢离开火车是无法跑动的，同理火车也可以有多节车厢。多线程的出现就是为了提高效率。同时它的出现也带来了一些问题。

## threading模块介绍：
`threading`模块是`python`中专门提供来做多线程的模块。`threading`模块中最常用的类是`Thread`。以下看一个简单的多线程程序：

In [1]:
# 传统方式
import threading
import time

def coding():
    for x in range(3):
        print('正在写代码%s' % x)
        time.sleep(1)
        
def drawing():
    for x in range(3):
        print('正在画图%s' % x)
        time.sleep(1)
        
def single_thread():
    coding()
    drawing()
    end = time.time()

In [2]:
single_thread()

正在写代码0
正在写代码1
正在写代码2
正在画图0
正在画图1
正在画图2


In [3]:
# 采用多线程的方式
def multi_thread():
    t1 = threading.Thread(target=coding)
    t2 = threading.Thread(target=drawing)
    
    t1.start()
    t2.start()
    

In [4]:
multi_thread()

正在写代码0正在画图0

正在画图1正在写代码1

正在画图2正在写代码2



## 查看线程数：
使用`threading.enumerate()`函数便可以看到当前线程的数量。

In [5]:
def multi_thread():
    t1 = threading.Thread(target=coding)
    t2 = threading.Thread(target=drawing)
    
    t1.start()
    t2.start()
    
    print(threading.enumerate())

multi_thread()
# 有三个线程

正在写代码0
正在画图0[<_MainThread(MainThread, started 140736248112000)>, <Thread(Thread-2, started daemon 123145409282048)>, <Heartbeat(Thread-3, started daemon 123145414537216)>, <HistorySavingThread(IPythonHistorySavingThread, started 123145420865536)>, <ParentPollerUnix(Thread-1, started daemon 123145426120704)>, <Thread(Thread-6, started 123145431375872)>, <Thread(Thread-7, started 123145436631040)>]

正在写代码1正在画图1

正在写代码2正在画图2



## 查看当前线程数的名字：
使用`threading.current_thread()`函数可以看到当前线程的信息。

In [6]:
def coding():
    for x in range(3):
        print('正在写代码%s' % threading.current_thread())
        time.sleep(1)
        
def drawing():
    for x in range(3):
        print('正在画图%s' % threading.current_thread())
        time.sleep(1)

def multi_thread():
    t1 = threading.Thread(target=coding)
    t2 = threading.Thread(target=drawing)
    
    t1.start()
    t2.start()

multi_thread()

正在写代码<Thread(Thread-8, started 123145431375872)>
正在画图<Thread(Thread-9, started 123145436631040)>
正在写代码<Thread(Thread-8, started 123145431375872)>正在画图<Thread(Thread-9, started 123145436631040)>

正在画图<Thread(Thread-9, started 123145436631040)>正在写代码<Thread(Thread-8, started 123145431375872)>



## 继承自`threading.Thread` 类：
为了让线程代码更好地封装。可以使用`threading`模块下的`Thread`类，继承自这个类，然后实现`run`方法，线程就会自动运行`run`方法中的代码。

In [7]:
class CodingThread(threading.Thread):
    def run(self):
            for x in range(3):
                print('正在写代码%s' % threading.current_thread())
                time.sleep(1)

class DrawingThread(threading.Thread):
    def run(self):
            for x in range(3):
                print('正在画图%s' % threading.current_thread())
                time.sleep(1)

def main():
    t1 = CodingThread()
    t2 = DrawingThread()
    
    t1.start()
    t2.start()

main()

正在写代码<CodingThread(Thread-10, started 123145431375872)>
正在画图<DrawingThread(Thread-11, started 123145436631040)>
正在写代码<CodingThread(Thread-10, started 123145431375872)>
正在画图<DrawingThread(Thread-11, started 123145436631040)>
正在写代码<CodingThread(Thread-10, started 123145431375872)>
正在画图<DrawingThread(Thread-11, started 123145436631040)>


## 多线程共享全局变量的问题：
多线程都是同一个进程中运行的。因此在进程中的全局变量所有线程都是可以共享的。这就造成了一个问题，因为现场执行的顺序是无序的。有可能造成数据错误。比如以下代码：

In [8]:
VALUE = 0
def add_value():
    global VALUE
    for x in range(1000000): #10, 100, 1000就不会出错 
        VALUE += 1
    print('value: %d' % VALUE)
        
def main():
    for x in range(2):
        t = threading.Thread(target=add_value)
        t.start()
        
main()

# 结果出错了
# 那怎么办？ -> 锁机制

value: 877247
value: 1156741


## 锁机制（用于会修改全局变量的代码）

In [9]:
VALUE = 0

gLock = threading.Lock()

def add_value():
    global VALUE
    gLock.acquire() #上锁
    for x in range(1000000): #10, 100, 1000就不会出错 
        VALUE += 1
    gLock.release() #解锁
    print('value: %d' % VALUE)
        
def main():
    for x in range(2):
        t = threading.Thread(target=add_value)
        t.start()
        
main()

value: 1000000
value: 2000000


## Lock版本生产者和消费者模式：
`生产者（多线程） -> 全局变量 -> 消费者（多线程）`  
生产者和消费者模式是多线程开发中常见到的一种模式。生产者的线程专门用来生成一些数据，然后存放到一个中介的变量中。消费者再从这个中间的变量中取出数据进行消费。但是因为要使用中间变量，中间变量经常是一些全局变量，因此需要使用锁来保证数据完整性。以下是使用`threading.Lock`锁实现的“生产者与消费者模式”的一个例子：

In [12]:
import threading
import random
import time

gMoney = 1000
gLock = threading.Lock()
gTotalTimes = 10
gTimes = 0

class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gTimes
        while True:
            money = random.randint(100, 1000)
            gLock.acquire()
            if gTimes >= gTotalTimes:
                gLock.release()
                break
            gMoney += money
            print('%s生产了%d元钱，剩余%d元钱'% (threading.current_thread(),
                                      money, gMoney))
            gTimes += 1
            gLock.release()
            time.sleep(0.5) #防止看起来太乱了
    
class Consumer(threading.Thread):
    def run(self):
        global gMoney
        while True:
            money = random.randint(100, 1000)
            gLock.acquire()
            if gMoney >= money:
                gMoney -= money
                print('%s消费了%d元钱，剩余%d元钱'% (threading.current_thread(),
                                            money, gMoney))
            else:
                if gTimes >= gTotalTimes:
                    gLock.release()
                    break
                print('%s消费者准备消费%d元钱，剩余%d元钱，不足！' % (
                threading.current_thread(), money, gMoney))
            gLock.release()
            time.sleep(0.5)

def main():
    for x in range(3):
        t = Consumer(name="消费者线程%d" % x)
        t.start()
    for x in range(5):
        t = Producer(name="生产者线程%d" % x)
        t.start()

In [13]:
main()

<Consumer(消费者线程0, started 123145431375872)>消费了409元钱，剩余591元钱
<Consumer(消费者线程1, started 123145436631040)>消费者准备消费910元钱，剩余591元钱，不足！
<Consumer(消费者线程2, started 123145441886208)>消费了428元钱，剩余163元钱
<Producer(生产者线程0, started 123145447141376)>生产了535元钱，剩余698元钱
<Producer(生产者线程1, started 123145452396544)>生产了334元钱，剩余1032元钱
<Producer(生产者线程2, started 123145457651712)>生产了236元钱，剩余1268元钱
<Producer(生产者线程3, started 123145462906880)>生产了147元钱，剩余1415元钱
<Producer(生产者线程4, started 123145468162048)>生产了961元钱，剩余2376元钱
<Consumer(消费者线程0, started 123145431375872)>消费了531元钱，剩余1845元钱
<Consumer(消费者线程1, started 123145436631040)>消费了946元钱，剩余899元钱
<Consumer(消费者线程2, started 123145441886208)>消费了737元钱，剩余162元钱
<Producer(生产者线程0, started 123145447141376)>生产了389元钱，剩余551元钱
<Producer(生产者线程1, started 123145452396544)>生产了767元钱，剩余1318元钱
<Producer(生产者线程2, started 123145457651712)>生产了325元钱，剩余1643元钱
<Producer(生产者线程3, started 123145462906880)>生产了504元钱，剩余2147元钱
<Producer(生产者线程4, started 123145468162048)>生产了296元钱，剩余2443元钱
<Consumer(消费者线程0, start

## Condition版的生产者与消费者模式：
`Lock`版本的生产者与消费者模式可以正常运行。但是存在一个不足，在消费者中，总是通过该`while True`死循环并且上锁的方式去判断钱够不够。上锁是一个很耗费CPU资源的行为。因此这种方式不是最好的。还有一种更好的方式便是使用`threading.Condition`来实现。`threading.Condition`可以再没有数据的时候处于阻塞的等待状态。一旦有核实的数据了，还可以使用`notify`相关函数来统治其他处于等待的线程。这样就可以不用做一些无用的上锁和解锁的操作。可以提高程序的性能。首先对`threading.Condition`相关的函数做个介绍,`threading.Condition`类似(继承于）`threading.Lock`，可以再修改全局数据的时候进行上锁，也可以在修改完毕后进行解锁。以下将一些常用的函数做个简单介绍：
1. `acquire`: 上锁
2. `release`: 解锁
3. `wait`: 将当前线程处于等待状态，并且会释放锁。可以被其他线程使用`notify`和`notify_all`函数唤醒。被唤醒后将继续等待上锁，上锁后继续执行下面的代码。
4. `notify`: 统治某个正在等待的线程，默认是第一个等待的线程。
5. `notify_all`: 统治所有正在等待的线程。`notify`和`notify`不会释放所。并且需要在`release`之前调用。  
以下是使用`threading.Condition`锁实现的“生产者与消费者模式”的一个例子：

In [14]:
import threading
import random
import time

gMoney = 1000
gCondition = threading.Condition()
gTotalTimes = 10
gTimes = 0

class Producer(threading.Thread):
    def run(self):
        global gMoney
        global gTimes
        while True:
            money = random.randint(100, 1000)
            gCondition.acquire()
            if gTimes >= gTotalTimes:
                gCondition.release()
                break
            gMoney += money
            print('%s生产了%d元钱，剩余%d元钱'% (threading.current_thread(),
                                      money, gMoney))
            gTimes += 1
            gCondition.notify_all() #唤醒所有
            gCondition.release()
            time.sleep(0.5)
    
class Consumer(threading.Thread):
    def run(self):
        global gMoney
        while True:
            money = random.randint(100, 1000)
            gCondition.acquire()
            while gMoney < money: #这个很重要，if的话，苏醒后再等待后，钱又不够了
                if gTimes >= gTotalTimes:
                    gCondition.release()
                    return #将整个函数立马返回，两个while都会被break
                print('%s消费者准备消费%d元钱，剩余%d元钱，不足！' % (
                threading.current_thread(), money, gMoney))
                gCondition.wait() #等待
            gMoney -= money
            print('%s消费了%d元钱，剩余%d元钱'% (threading.current_thread(),
                                            money, gMoney))
            gCondition.release()
            time.sleep(0.5)

def main():
    for x in range(3):
        t = Consumer(name="消费者线程%d" % x)
        t.start()
    for x in range(5):
        t = Producer(name="生产者线程%d" % x)
        t.start()

In [16]:
main()

<Consumer(消费者线程0, started 123145431375872)>消费了406元钱，剩余594元钱
<Consumer(消费者线程1, started 123145436631040)>消费者准备消费797元钱，剩余594元钱，不足！
<Consumer(消费者线程2, started 123145441886208)>消费了375元钱，剩余219元钱
<Producer(生产者线程0, started 123145447141376)>生产了736元钱，剩余955元钱
<Consumer(消费者线程1, started 123145436631040)>消费了797元钱，剩余158元钱
<Producer(生产者线程1, started 123145452396544)>生产了433元钱，剩余591元钱
<Producer(生产者线程2, started 123145457651712)>生产了260元钱，剩余851元钱
<Producer(生产者线程3, started 123145462906880)>生产了601元钱，剩余1452元钱
<Producer(生产者线程4, started 123145468162048)>生产了305元钱，剩余1757元钱
<Consumer(消费者线程0, started 123145431375872)>消费了581元钱，剩余1176元钱
<Consumer(消费者线程2, started 123145441886208)>消费了583元钱，剩余593元钱
<Producer(生产者线程0, started 123145447141376)>生产了133元钱，剩余726元钱
<Consumer(消费者线程1, started 123145436631040)>消费了548元钱，剩余178元钱
<Producer(生产者线程1, started 123145452396544)>生产了279元钱，剩余457元钱
<Producer(生产者线程2, started 123145457651712)>生产了328元钱，剩余785元钱
<Producer(生产者线程3, started 123145462906880)>生产了842元钱，剩余1627元钱
<Producer(生产者线程4, started 12

## Queue线程安全队列：
在线程中，访问一些全局变量，加锁是一个经常的过程。如果你是想把一些数据存储到某个队列中，那么Pyhton内置了一个线程安全的模块叫做`queue`模块。Python中的`queue`模块中提供了同步的、线程安全的队列类，包括FIFO(先进先出）队列Queue，LIFO（后入先出）队列LifoQueue。这些队列都实现了锁原语（可以理解为原子操作，即要么不做，要么都做完），能够在多线程中直接使用。可以使用队列来实现线程的同步。相关的函数如下：
1. 初始化Queue(maxsize): 创建一个先进先出的队列。
2. qsize(): 返回队列的大小。
3. empty(): 判断队列是否为空。
4. full(): 判断队列是否满了。
5. get(): 从队列中取最后一个数据。
6. put(): 将一个数据放在队列中。

In [28]:
from queue import Queue

q = Queue(4)
q.put(1)
q.put(2)

print(q.qsize())
print(q.empty())
print(q.full())
print(q.get())

2
False
False
1


In [29]:
q = Queue(4)

for x in range(4):
    q.put(x)

for x in range(4):
    print(q.get()) #先进先出

0
1
2
3


In [35]:
# block操作
# q.get(block=True) #如果队列中没有值，会一直阻塞
# q.put(block=True) #如果队列中满了，会一直阻塞

def set_value(q):
    index = 0
    while True:
        q.put(index)
        index += 1
        time.sleep(3)
        
def get_value(q):
    while True:
        print(q.get())

def main():
    q = Queue(4)
    t1 = threading.Thread(target=set_value, args=[q]) #传递参数
    t2 = threading.Thread(target=get_value, args=[q]) #传递参数
    
    t1.start()
    t2.start()

In [36]:
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
